In [ ]:
# ComfyUI Batch Image Generator for Real Estate Business

This notebook will:
1. Install ComfyUI on Google Colab
2. Download your models  
3. Generate hundreds of product images
4. Package them for download

**Much faster than your Mac!**


In [ ]:
# Install ComfyUI and Dependencies
print("Installing ComfyUI...")

!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
!pip install -r requirements.txt
!pip install requests pillow

print("ComfyUI installed!")


In [ ]:
# Download AI Models
print("Downloading Stable Diffusion model...")

import os
os.makedirs('models/checkpoints', exist_ok=True)

# Download the model you were using
!wget -O models/checkpoints/v1-5-pruned-emaonly-fp16.safetensors \
  https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors

print("Model downloaded!")
!ls -la models/checkpoints/


In [ ]:
# 🎨 Generate Real Estate Product Images

# Define your product prompts
product_prompts = [
    # Business Cards
    "Professional business cards, premium quality, clean white background, studio lighting, commercial photography",
    "Luxury business cards with metallic finish, high-end presentation, professional lighting",
    "Corporate business cards, standard quality, clean design, white background",
    "Real estate business cards, elegant design, professional presentation",
    "Premium business cards with embossed elements, luxury finish",
    
    # Flyers
    "Real estate flyers, professional design, property marketing materials, clean layout",
    "Property listing flyers, high-quality print materials, professional presentation",
    "Real estate marketing flyers, modern design, commercial photography",
    "Luxury property flyers, premium presentation, elegant design",
    "Open house flyers, professional real estate marketing materials",
    
    # Banners
    "Real estate banners, professional signage, outdoor advertising materials",
    "Property sale banners, high-quality vinyl printing, commercial grade",
    "Real estate yard signs, professional presentation, durable materials",
    "Open house banners, eye-catching design, professional quality",
    "Real estate advertising banners, premium materials, weather resistant",
    
    # Brochures
    "Real estate brochures, tri-fold design, professional presentation",
    "Property brochures, high-quality printing, elegant layout",
    "Luxury real estate brochures, premium paper stock, professional design",
    "Real estate marketing brochures, informational layout, clean design",
    "Property listing brochures, professional presentation, detailed information"
]

print(f"🎯 Ready to generate {len(product_prompts)} product images...")
print("📝 Run the next cell to start generation!")


In [ ]:
# Start ComfyUI Server
import subprocess
import time
import threading

def start_comfyui():
    subprocess.run(['python', 'main.py', '--listen', '0.0.0.0', '--port', '8188'])

# Start ComfyUI in background
thread = threading.Thread(target=start_comfyui, daemon=True)
thread.start()

print("Starting ComfyUI server...")
time.sleep(30)  # Wait for startup

# Test if server is running
import requests
try:
    response = requests.get('http://localhost:8188')
    print("ComfyUI server is running!")
except:
    print("Server might still be starting...")


In [ ]:
# Batch Image Generator Class
import json
import requests
import time
import os
from pathlib import Path

class ColabBatchGenerator:
    def __init__(self):
        self.base_url = 'http://localhost:8188'
        self.output_dir = Path('output')
        self.generated_count = 0
    
    def create_workflow(self, prompt, seed=None):
        if seed is None:
            seed = int(time.time()) + self.generated_count
        
        return {
            "1": {
                "inputs": {"ckpt_name": "v1-5-pruned-emaonly-fp16.safetensors"},
                "class_type": "CheckpointLoaderSimple"
            },
            "2": {
                "inputs": {"text": prompt, "clip": ["1", 1]},
                "class_type": "CLIPTextEncode"
            },
            "3": {
                "inputs": {
                    "text": "blurry, low quality, text, watermark",
                    "clip": ["1", 1]
                },
                "class_type": "CLIPTextEncode"
            },
            "4": {
                "inputs": {
                    "seed": seed,
                    "steps": 10,  # Fast generation
                    "cfg": 7.0,
                    "sampler_name": "euler",
                    "scheduler": "normal",
                    "denoise": 1.0,
                    "model": ["1", 0],
                    "positive": ["2", 0],
                    "negative": ["3", 0],
                    "latent_image": ["5", 0]
                },
                "class_type": "KSampler"
            },
            "5": {
                "inputs": {"width": 512, "height": 512, "batch_size": 1},
                "class_type": "EmptyLatentImage"
            },
            "6": {
                "inputs": {"samples": ["4", 0], "vae": ["1", 2]},
                "class_type": "VAEDecode"
            },
            "7": {
                "inputs": {
                    "filename_prefix": f"colab_product_{self.generated_count:04d}",
                    "images": ["6", 0]
                },
                "class_type": "SaveImage"
            }
        }
    
    def generate_batch(self, prompts):
        print(f"Generating {len(prompts)} images...")
        
        for i, prompt in enumerate(prompts):
            workflow = self.create_workflow(prompt, seed=i+1000)
            
            try:
                # Queue prompt
                response = requests.post(f'{self.base_url}/prompt', 
                                       json={'prompt': workflow})
                
                if response.status_code == 200:
                    prompt_id = response.json()['prompt_id']
                    print(f"  Queued {i+1}/{len(prompts)}: {prompt[:40]}...")
                    
                    # Wait for completion
                    while True:
                        hist_response = requests.get(f'{self.base_url}/history/{prompt_id}')
                        if hist_response.status_code == 200:
                            history = hist_response.json()
                            if prompt_id in history:
                                print(f"  Completed {i+1}/{len(prompts)}")
                                break
                        time.sleep(2)
                    
                    self.generated_count += 1
                else:
                    print(f"  Failed to queue prompt {i+1}")
                    
            except Exception as e:
                print(f"  Error with prompt {i+1}: {e}")
        
        print(f"Batch complete! Generated {len(prompts)} images")

# Initialize generator
generator = ColabBatchGenerator()
print("Batch generator ready!")


In [ ]:
# Generate Real Estate Product Images
product_prompts = [
    # Business Cards
    "Professional business cards, premium quality, clean white background, studio lighting, commercial photography",
    "Luxury business cards with metallic finish, high-end presentation, professional lighting",
    "Corporate business cards, standard quality, clean design, white background",
    "Real estate business cards, elegant design, professional presentation",
    "Premium business cards with embossed elements, luxury finish",
    
    # Flyers
    "Real estate flyers, professional design, property marketing materials, clean layout",
    "Property listing flyers, high-quality print materials, professional presentation",
    "Real estate marketing flyers, modern design, commercial photography",
    "Luxury property flyers, premium presentation, elegant design",
    "Open house flyers, professional real estate marketing materials",
    
    # Banners
    "Real estate banners, professional signage, outdoor advertising materials",
    "Property sale banners, high-quality vinyl printing, commercial grade",
    "Real estate yard signs, professional presentation, durable materials",
    "Open house banners, eye-catching design, professional quality",
    "Real estate advertising banners, premium materials, weather resistant",
    
    # Brochures
    "Real estate brochures, tri-fold design, professional presentation",
    "Property brochures, high-quality printing, elegant layout",
    "Luxury real estate brochures, premium paper stock, professional design",
    "Real estate marketing brochures, informational layout, clean design",
    "Property listing brochures, professional presentation, detailed information"
]

# Generate images
print(f"Starting generation of {len(product_prompts)} product images...")
start_time = time.time()

generator.generate_batch(product_prompts)

end_time = time.time()
total_time = end_time - start_time

print(f"GENERATION COMPLETE!")
print(f"Total time: {total_time/60:.1f} minutes")
print(f"Average: {total_time/len(product_prompts):.1f} seconds per image")
print(f"Images saved to: output/")


In [ ]:
# Package Images for Download
import zipfile
import glob

print("Creating download package...")

# Create ZIP file with all generated images
with zipfile.ZipFile('real_estate_ai_images.zip', 'w') as zipf:
    # Find all generated images
    image_files = glob.glob('output/*.png')
    
    for image_file in image_files:
        # Add to zip with clean filename
        filename = os.path.basename(image_file)
        zipf.write(image_file, filename)
        print(f"  Added: {filename}")

print(f"Package created: real_estate_ai_images.zip")
print(f"Total images: {len(image_files)}")

# Show download link
from google.colab import files
print("Click below to download your images:")
files.download('real_estate_ai_images.zip')
